# How to get the admittance matrix Ybus and the Jacobian matrix J from a pandapower net


This jupyter notebooks explains how to obtain power flow matrices from a pandapower grid. These two matrices will be extracted:
* Ybus = admittance matrix
* J = jacobian matrix in dependency of the voltage vector

We'll examine an example grid (IEEE case9), but this method should work with any other grid as well.

## How to get Ybus
Ybus is stored after a power flow, since it can be "recycled" in a new power flow to save calculation time. This makes it easy to obtain it

In [ ]:
import pandapower as pp
import pandapower.networks as nw

def get_Ybus(net):
    """
    This functions returns the admittance matrix (Ybus) from internal pandapower calculations
    
    """
    # run a power flow to get the ppc (which contains Ybus afterwards)
    pp.runpp(net)

    # ppc (this is similar to the PYPOWER format file, with some changes) contains Ybus
    ppc = net._ppc

    # Get stored Ybus in ppc
    Ybus = ppc["internal"]["Ybus"]
    
    return Ybus


if __name__ == "__main__":
    # load example network of which you want the Ybus
    net = nw.case9()
    Ybus = get_Ybus(net)
    print("This is Ybus:\n")
    print(Ybus)
    

# How to get J
J is not stored in net since it is generated in every Newton-Raphson iteratione depending on the current voltage vector. This makes it a litlle bit harder to obtain.

In [ ]:
import pandapower as pp
import pandapower.networks as nw

# functions we need for creating J
import numpy as np
from pandapower.pf.run_newton_raphson_pf import _get_pf_variables_from_ppci
from pandapower.pd2ppc import _pd2ppc
from pandapower.pf.newtonpf import _create_J_without_numba


def get_J(net, V=None):

    _, ppci = _pd2ppc(net)
    baseMVA, bus, gen, branch, ref, pv, pq, _, _, V0 = _get_pf_variables_from_ppci(ppci)
        
    Ybus = get_Ybus(net)
    if V is None:
        V = V0
    Ibus = np.zeros(len(V))
    pvpq = np.r_[pv, pq]
    
    J = _create_J_without_numba(Ybus, V, pvpq, pq, Ibus=Ibus)
    
    return J
    

if __name__ == "__main__":
    # load example network of which you want the Ybus and J
    net = nw.case9()
    
    J = get_J(net)
    print("This is J with V0;\n")
    print(J)